Data Loading

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:


df=pd.read_csv('/home/jimbrootan/Desktop/bbc/bbc_news.csv')
df.tail()
df['Category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: Category, dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  2225 non-null   object
 1   News      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [4]:
df.isnull().sum()

Category    0
News        0
dtype: int64

In [5]:
import nltk
from nltk.corpus import stopwords
stopword=list(stopwords.words('english'))

In [6]:

import re
stopword = set(stopwords.words('english'))
# print(stopword)
def clear_text(text):
    #reference: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = text.replace('\\r', ' ')
    text = text.replace('\\"', ' ')
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')
    text = ' '.join(word for word in text.split() if word not in stopword)

    text = re.sub('[^A-Za-z0-9]+', ' ', text)
#     text = text.lower()
    return text
    
processed_titles= []
for title in df['News'].values:
    processed_title = clear_text(title)
    processed_titles.append(processed_title)


In [7]:
#tokenizing  news 
tokenizer = Tokenizer(num_words = 1000, oov_token="<OOV>")
tokenizer.fit_on_texts(processed_titles)
X = tokenizer.word_index
# Labels
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(df['Category'])

label_seq = label_tokenizer.texts_to_sequences(df['Category'])
y=label_seq

In [8]:
#splitting train and pest part

train_size = int(len(processed_titles) * 0.8)
X_train=processed_titles[:train_size]
X_test=processed_titles[train_size:]
y_train=df["Category"][:train_size]
y_test=df["Category"][train_size:]





#define Tokenizer ,refernce: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer


In [9]:
#For x_train,  X_test

word_tokenizer=Tokenizer(
    num_words=1000,oov_token=None
    )

word_tokenizer.fit_on_texts(processed_titles)
index_word = tokenizer.word_index
X_train_sequences = word_tokenizer.texts_to_sequences(X_train)
X_test_sequences=word_tokenizer.texts_to_sequences(X_test)


In [10]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(y)
y_train_sequences = np.array(label_tokenizer.texts_to_sequences(y_train))
y_test_sequences = np.array(label_tokenizer.texts_to_sequences(y_test))




AttributeError: 'int' object has no attribute 'lower'

In [ ]:
#creating the model


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000,16, input_length=150),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')
])

In [ ]:
#compiling the model

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()



In [ ]:
#fitting the modeabsl

num_epochs = 30
history = model.fit(X_train_sequences,y_train_sequences, epochs=num_epochs, 
                    validation_data=(X_train_sequences,y_train_sequences), verbose=2)